In [ ]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import utils
from joblib import load
import numpy as np
import seaborn as sns
from sklearn.metrics import (
    accuracy_score, f1_score, precision_score, recall_score, confusion_matrix, classification_report, ConfusionMatrixDisplay
)
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 30)
plt.style.use('ggplot')

In [ ]:
X_test, y_test = utils.load_dataset("../dataset/test.csv")
X_test, _ = utils.scale_data(X_test, X_test)

# shuffle the data
X_test = X_test.sample(frac=1, random_state=42)
y_test = y_test.sample(frac=1, random_state=42)

In [ ]:
model_avg = load('../model/model_fedavg.joblib')
model_median = load('../model/model_fedmedian.joblib')
model_trimmed = load('../model/model_fedtrimmedavg.joblib')

y_pred_avg = model_avg.predict(X_test)
y_pred_median = model_median.predict(X_test)
y_pred_trimmed = model_trimmed.predict(X_test)

In [ ]:
print(classification_report(y_test, y_pred_avg))
print('='*50)
print(classification_report(y_test, y_pred_median))
print('='*50)
print(classification_report(y_test, y_pred_trimmed))

In [ ]:
def generate_report(y_test, y_pred, title):
    acc_score = accuracy_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred, average=None)
    precision = precision_score(y_test, y_pred, average=None)
    recall = recall_score(y_test, y_pred, average=None)

    print(f"Accuracy: {acc_score:.2%}\n\
    F1 Score: {dict(zip(model_avg.classes_, map(lambda x: f'{x:.2%}', f1)))}\n\
    Precision Score: {dict(zip(model_avg.classes_, map(lambda x: f'{x:.2%}', precision)))}\n\
    Recall Score: {dict(zip(model_avg.classes_, map(lambda x: f'{x:.2%}', recall)))}\n\
    ")

    cm = confusion_matrix(y_test, y_pred)
    cm_norm = confusion_matrix(y_test, y_pred, normalize="true")
    annot = np.array([f"{i}\n({g:.1%})" for i, g in zip(cm.flatten(), cm_norm.flatten())])
    annot = annot.reshape(cm.shape)

    fig = plt.figure(dpi=90)
    sns.heatmap(cm, annot=annot, fmt="", xticklabels=["Poor", "Standard", "Good"], yticklabels=["Poor", "Standard", "Good"])
    plt.title(title)
    plt.ylabel('Actual')
    plt.xlabel('Predicted')
    plt.show()

In [ ]:
generate_report(y_test, y_pred_avg, "FedAvg")
generate_report(y_test, y_pred_median, "FedMedian")
generate_report(y_test, y_pred_trimmed, "FedTrimmedAvg")